In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Key point detection using Mediapipe


In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 33.9 MB/s eta 0:00:00


In [ ]:
import cv2
import os
from PIL import Image
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
def final_output(path):
  with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
      image=cv2.imread(path)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      image.flags.writeable = False

      # Make detection
      results = pose.process(image)

      # Recolor back to BGR
      image.flags.writeable = True
      image = cv2.cvtColor(np.zeros_like(image), cv2.COLOR_RGB2BGR)
      landmarks=results.pose_landmarks.landmark
      return landmarks

In [ ]:
import os
import random as r
files = []
for dirname, dir, filenames in os.walk('/content/drive/MyDrive/YogaDataset/TRAIN'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))

In [ ]:
files_test = []
for dirname, dir, filenames in os.walk('/content/drive/MyDrive/YogaDataset/TEST'):
    for filename in filenames:
        files_test.append(os.path.join(dirname, filename))

In [ ]:
key_points_train={}
for path in files:
  keypoint=[]
  try:
    landmarks=final_output(path)
    for marks in landmarks:
      keypoint.append(marks.x)
      keypoint.append(marks.y)
    key_points_train[path]=keypoint
  except:
    print(path) #Mediapipe is not able to detect landmarks on these images

/content/drive/MyDrive/YogaDataset/TRAIN/warrior2/00000137.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/warrior2/00000182.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/warrior2/00000189.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/warrior2/00000326.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/downdog/00000147.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/downdog/00000188.png
/content/drive/MyDrive/YogaDataset/TRAIN/downdog/00000173.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/downdog/00000168.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/downdog/00000212.png
/content/drive/MyDrive/YogaDataset/TRAIN/downdog/00000224.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/downdog/00000221.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/downdog/00000235.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/downdog/00000220.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/downdog/00000232.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/downdog/00000239.jpg
/content/drive/MyDrive/YogaDataset/TRAIN/downdog/00000270.jpg
/con

In [ ]:
key_points_test={}
paths=[]
for path in files_test:
  keypoint=[]
  try:
    landmarks=final_output(path)
    for marks in landmarks:
      keypoint.append(marks.x)
      keypoint.append(marks.y)
      key_points_test[path]=keypoint
  except:
    print(path)   #Mediapipe cannot predict landmarks on these test data.
    paths.append(path)

/content/drive/MyDrive/YogaDataset/TEST/warrior2/00000025.png
/content/drive/MyDrive/YogaDataset/TEST/warrior2/00000039.png
/content/drive/MyDrive/YogaDataset/TEST/downdog/00000010.png
/content/drive/MyDrive/YogaDataset/TEST/downdog/00000080.jpg
/content/drive/MyDrive/YogaDataset/TEST/downdog/00000120.jpg


In [ ]:
import pandas as pd
key_points_train_df = pd.DataFrame(columns=["key_points","label"])
key_points_test_df = pd.DataFrame(columns=["key_points","label"])

In [ ]:
label = []
for img in key_points_train.keys():
    label.append(img.split('/')[6])

In [ ]:
key_points_train_df.loc[:,"key_points"]=key_points_train.values() #only assigning the images on which landmarks are detected
key_points_train_df.loc[:,"label"]=label

In [ ]:
one_hot = pd.get_dummies(key_points_train_df['label'], prefix='label')
key_points_train_df = pd.concat([key_points_train_df, one_hot], axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
key_points_train_df['label_code'] = le.fit_transform(key_points_train_df['label'])

In [ ]:
key_points_train_df.drop(columns=['label'],inplace=True)

,key_points,label_downdog,label_goddess,label_plank,label_tree,label_warrior2
0,"[0.364690899848938, 0.1572379767894745, 0.3620...",0,0,0,0,1
1,"[0.44384700059890747, 0.28754690289497375, 0.4...",0,0,0,0,1
2,"[0.44388651847839355, 0.20423871278762817, 0.4...",0,0,0,0,1
3,"[0.44745945930480957, 0.3644116222858429, 0.45...",0,0,0,0,1
4,"[0.5981495380401611, 0.22829404473304749, 0.58...",0,0,0,0,1
...,...,...,...,...,...,...
1035,"[0.48942673206329346, 0.20805078744888306, 0.5...",0,1,0,0,0
1036,"[0.5059691071510315, 0.3123514652252197, 0.516...",0,1,0,0,0
1037,"[0.519749104976654, 0.3131095767021179, 0.5255...",0,1,0,0,0
1038,"[0.4950077533721924, 0.24896344542503357, 0.50...",0,1,0,0,0


In [ ]:
new_data = key_points_train_df['key_points'].apply(pd.Series)
key_points_train_df = pd.concat([new_data,key_points_train_df],axis=1)

In [ ]:
key_points_train_df=key_points_train_df.drop(columns=['key_points'])
# These dataframe have 66 keypoints column and label_encoder and One hot encoder to train for both Random Forest and Neural Network

In [ ]:
import tensorflow as tf
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=66, activation='relu'))   #input layer
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))   #hidden layer
ann.add(tf.keras.layers.Dense(units=5, activation='softmax')) #output layer with softmax function

In [ ]:
ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
y = key_points_train_df.iloc[:,66:-1]
X = key_points_train_df.iloc[:,:66]
ann.fit(X, y, batch_size = 1, epochs = 50)

Epoch 1/50
1040/1040 [==============================] - 2s 2ms/step - loss: 0.2477 - accuracy: 0.9106
Epoch 2/50
1040/1040 [==============================] - 2s 2ms/step - loss: 0.2324 - accuracy: 0.9212
Epoch 3/50
1040/1040 [==============================] - 3s 2ms/step - loss: 0.2391 - accuracy: 0.9154
Epoch 4/50
1040/1040 [==============================] - 3s 2ms/step - loss: 0.2240 - accuracy: 0.9202
Epoch 5/50
1040/1040 [==============================] - 2s 2ms/step - loss: 0.2046 - accuracy: 0.9260
Epoch 6/50
1040/1040 [==============================] - 2s 2ms/step - loss: 0.2113 - accuracy: 0.9308
Epoch 7/50
1040/1040 [==============================] - 2s 2ms/step - loss: 0.2162 - accuracy: 0.9231
Epoch 8/50
1040/1040 [==============================] - 2s 2ms/step - loss: 0.2098 - accuracy: 0.9298
Epoch 9/50
1040/1040 [==============================] - 2s 2ms/step - loss: 0.1962 - accuracy: 0.9327
Epoch 10/50
1040/1040 [==============================] - 2s 2ms/step - loss: 0.189

In [ ]:
from sklearn.ensemble import RandomForestClassifier
y_rf = key_points_train_df['label_code'] #Column with Label encoder
clf = RandomForestClassifier(random_state = 42,
                             n_jobs = 1,
                             n_estimators = 100)

In [ ]:
clf.fit(X,y_rf)

In [ ]:
label_t = []
for img in key_points_test.keys():
    label_t.append(img.split('/')[6])

In [ ]:
key_points_test_df.loc[:,"key_points"]=key_points_test.values()
key_points_test_df.loc[:,"label"]=label_t
key_points_test_df['label_code'] = le.transform(key_points_test_df['label'])

In [ ]:
new_data_test = key_points_test_df['key_points'].apply(pd.Series)
key_points_test_df = pd.concat([new_data_test,key_points_test_df['label_code']],axis=1)

In [ ]:
y_test = key_points_test_df['label_code']
X_test = key_points_test_df.drop('label_code',axis=1)

In [ ]:
prediction = ann.predict(X_test) # prediction with neural network model
pred_rf=clf.predict(X_test)      #prediction with random forest classifier

15/15 [==============================] - 0s 1ms/step


In [ ]:
pred=np.argmax(prediction,axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy with NN:",accuracy_score(y_test,pred))
print("Accuracy with RFC:",accuracy_score(y_test,pred_rf))

Accuracy: 0.9621052631578947


#Keypoints detection using YOLOV7 model


In [ ]:
%cd /content/drive/MyDrive/yolov7

In [ ]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import torch
import cv2
from torchvision import transforms
import numpy as np
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

In [ ]:
def load_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load('yolov7-w6-pose.pt', map_location=device)['model']
    model.float().eval()
    if torch.cuda.is_available():
        model.to(device)
    return model


In [ ]:
import gc
key_points_dict = {}
for img in files:
    with torch.no_grad():
        ## Transform image from numpy to torch format
        image = cv2.imread(img)
        image = letterbox(image, 960, stride=64, auto=True)[0]
        image = transforms.ToTensor()(image)
        image = torch.tensor(np.array([image.numpy()]))

        if torch.cuda.is_available():
            image = image.half().to(device)
        model=load_model()
        output, _ = model(image)
    output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
    out1 = output

    keypoints = output_to_keypoint(out1)
    # iterate for multiple figures in an image
    for idx in range(keypoints.shape[0]):
        keypoints_list = keypoints[idx,7:].tolist()
    key_points_dict[img] = keypoints_list
    del image, output
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
key_points_df = pd.DataFrame(columns=["key_points","label"])
key_points_test = pd.DataFrame(columns=["key_points","label"])

In [ ]:
label = []
for img in key_points_train.keys():
    label.append(img.split('/')[6])

In [ ]:
key_points_train_df.loc[:,"key_points"]=key_points_train.values() #only assigning the images on which landmarks are detected
key_points_train_df.loc[:,"label"]=label

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
key_points_df['label_code'] = le.fit_transform(key_points_df['label'])

In [ ]:
one_hot = pd.get_dummies(key_points_train_df['label'], prefix='label')
key_points_train_df = pd.concat([key_points_train_df, one_hot], axis=1)

In [ ]:
key_points_train_df=key_points_train_df.drop(columns=['label'])
key_points_train_df

In [ ]:
new_data = key_points_df['key_points'][:1081].apply(pd.Series)
key_points_df = pd.concat([new_data,key_points_df['label_code']],axis=1)

In [ ]:
key_points_train_df=key_points_train_df.drop(columns=['key_points'])
# These dataframe have 66 keypoints column and label_encoder and One hot encoder to train for both Random Forest and Neural Network

In [ ]:
import tensorflow as tf
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=66, activation='relu'))   #input layer
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))   #hidden layer
ann.add(tf.keras.layers.Dense(units=5, activation='softmax')) #output layer with softmax function

In [ ]:
ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
y = key_points_train_df.iloc[:,66:-1]
X = key_points_train_df.iloc[:,:66]
ann.fit(X, y, batch_size = 1, epochs = 50)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
y_rf = key_points_train_df['label_code'] #Column with Label encoder
clf = RandomForestClassifier(random_state = 42,
                             n_jobs = 1,
                             n_estimators = 100)

In [ ]:
clf.fit(X,y_rf)

In [ ]:
label_t = []
for img in key_points_test.keys():
    label_t.append(img.split('/')[6])

In [ ]:
key_points_test_df.loc[:,"key_points"]=key_points_test.values()
key_points_test_df.loc[:,"label"]=label_t
key_points_test_df['label'] = le.transform(key_points_test_df['label'])

In [ ]:
new_data_test = key_points_test_df['key_points'].apply(pd.Series)
key_points_test_df = pd.concat([new_data_test,key_points_test_df['label_code']],axis=1)

In [ ]:
y_test = key_points_test_df['label']
X_test = key_points_test_df.drop('label',axis=1)

In [ ]:
prediction = ann.predict(X_test) # prediction with neural network model
pred_rf=clf.predict(X_test)      #prediction with random forest classifier

In [ ]:
pred=np.argmax(prediction,axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy with NN:",accuracy_score(y_test,pred))
print("Accuracy with RFC:",accuracy_score(y_test,pred_rf))